# CNN(Convolutional Neural Network)
> 참고: 책 <<밑바닥부터 시작하는 딥러닝1>>, pp.242-253

In [4]:
import sys
sys.path.append('../')

import numpy as np
from common.util import im2col

합성곱 계층 구현

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        # filter shape: (출력 채널 수, 입력 채널 수, 높이, 너비)
        FN, C, FH, FW = self.W.shape

        # input data shape: (데이터 수, 채널 수, 높이, 너비)
        N, C, H, W = x.shape

        # 출력(feature map)의 높이와 너비
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        
        # 필터 전개
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b

        # transpose로 축의 순서를 바꿔준다.
        # (N, H, W, C) -> (N, C, H, W)
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out


풀링 계층 구현

In [9]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # 최대값
        out = np.max(col, axis=1)

        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out